In [1]:
sys.stderr = open(snakemake.log[0], "w")
import pysam
#import glob

#snakemake_input = glob.glob("../../results/benchmarking/assembly/*")
#snakemake_output = '../../conda.csv'

#with open(snakemake_output, "w") as out:
with open(snakemake.output[0], "w") as out:
   print("Accession", "Contigs", 'Total contigs', 'Contig length', 'Reference length', 'Contig frac', 'Cum frac','Edit dist.', 'Edit frac', sep="\t")
   print("Accession", "Contigs", 'Total contigs', 'Contig length', 'Reference length', 'Contig frac', 'Cum frac','Edit dist.', 'Edit frac', sep="\t", file=out)
   sum_of_edit_dist = 0
   largest_no_contig = 0
   small_larg_cover = 1.0
   small_larg_cover_name = 'asd'
   smallest_cum_frac = 1.0

   #for bam_file in snakemake_input:
   for bam_file in snakemake.input:
      current_contig = 1

      with pysam.AlignmentFile(bam_file, "rb") as samfile:
         total_contigs = samfile.count()
         accession = samfile.get_reference_name(0)
         
         largest_no_contig = total_contigs if total_contigs > largest_no_contig else largest_no_contig

      with pysam.AlignmentFile(bam_file, "rb") as samfile:
         ref_lengths = samfile.lengths[0]
         largest_coverage = 0
         cum_frac = 0
         for read in samfile.fetch():
            query_alignment_length = read.query_alignment_length
            frac = round(query_alignment_length / ref_lengths, 2)
            edit = read.get_tag("NM")
            cum_frac += frac
            cum_frac = round(cum_frac, 2)
            sum_of_edit_dist = sum_of_edit_dist + int(edit)
            edit_frac = round(read.get_tag("NM")/query_alignment_length, 5)
            largest_coverage = frac if frac > largest_coverage else largest_coverage

            print(accession, current_contig, total_contigs, query_alignment_length, ref_lengths, frac, cum_frac, edit, edit_frac, sep="\t")
            print(accession, current_contig, total_contigs, query_alignment_length, ref_lengths, frac, cum_frac, edit, edit_frac, sep="\t", file=out)

            current_contig += 1
         smallest_cum_frac = cum_frac if smallest_cum_frac>cum_frac else smallest_cum_frac
         small_larg_cover = largest_coverage if small_larg_cover>largest_coverage else small_larg_cover
         small_larg_cover_name = accession if small_larg_cover>=largest_coverage else small_larg_cover_name

   print('Largest number of contigs')
   print(largest_no_contig)
   print('Smallest largest coverage in ' + str(small_larg_cover_name))
   print(small_larg_cover)
   print('Sum of edit distances')
   print(sum_of_edit_dist)
   print('Smallest cum. fraction of contigs')
   print(smallest_cum_frac)
   print('Largest number of contigs', file=out)
   print(largest_no_contig, file=out)
   print('Smallest largest coverage in ' + str(small_larg_cover_name), file=out)
   print(small_larg_cover, file=out)
   print('Sum of edit distances', file=out)
   print(sum_of_edit_dist, file=out)
   print('Smallest cum. fraction of contigs', file=out)
   print(smallest_cum_frac, file=out)


NameError: name 'snakemake' is not defined